# Customer Churn Prediction Modelling

## Step 1: Import libraries

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from imblearn.over_sampling import SMOTE
from sklearn import preprocessing, ensemble  
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, make_scorer, precision_score, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  
from sklearn.naive_bayes import GaussianNB  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier  
from sklearn.ensemble import AdaBoostClassifier  
from sklearn.ensemble import ExtraTreesClassifier  
from sklearn.ensemble import BaggingClassifier  
from sklearn.ensemble import RandomForestClassifier  
from sklearn.svm import SVC

import keras
import keras_metrics
from keras.models import Sequential
from keras.layers import Dense

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


## Step 2: Read csv file and explore variables

In [ ]:
custchurn=pd.read_csv("C:\\Users\\wongannnee\\Google Drive\\ytd_churn_data.csv") 
pd.set_option('display.max_column',None)


In [ ]:
# 2-1:  Total records & variables, data types, unique values, missing values, first 5 records

def checktable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Variable'] = summary['index']
    summary = summary[['Variable','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['1st Record'] = df.loc[0].values
    summary['2nd Record'] = df.loc[1].values
    summary['3rd Record'] = df.loc[2].values
    summary['4th Record'] = df.loc[3].values
    summary['5th Record'] = df.loc[4].values
    return summary

checktable(custchurn)

In [6]:
# 2-2: Chech the number of yes and no for imbalanced classification target variable in'Churn'
pd.value_counts(custchurn.Churn)


No     5174
Yes    1869
Name: Churn, dtype: int64

### Step 3: Exploratory Data Analysis

In [7]:
#3-1: Change 'SeniorCitizen' from interger to categorical variable
custchurn['SeniorCitizen']=pd.Categorical(custchurn['SeniorCitizen'])
print(custchurn.dtypes)


customerID            object
gender                object
SeniorCitizen       category
Partner               object
Dependents            object
tenure                 int64
PhoneService          object
MultipleLines         object
InternetService       object
OnlineSecurity        object
OnlineBackup          object
DeviceProtection      object
TechSupport           object
StreamingTV           object
StreamingMovies       object
Contract              object
PaperlessBilling      object
PaymentMethod         object
MonthlyCharges       float64
TotalCharges         float64
Churn                 object
dtype: object


In [ ]:
# 3-2: Plot Numerical Variables - Histogram, Probablity Plot, Box plot

def diagnostic_plots(df, variable):
    
    plt.figure(figsize=(20, 5))

    plt.subplot(1, 3, 1)
    sns.distplot(df[variable], bins=30,kde_kws={'bw': 1.5})
    plt.title('Histogram')
    
    plt.subplot(1, 3, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    plt.ylabel('RM quantiles')

    plt.subplot(1, 3, 3)
    sns.boxplot(y=df[variable])
    plt.title('Boxplot')
    
    plt.show()
    

num_columns=custchurn.select_dtypes(exclude=["object","category"]).columns
for i in num_columns:
    diagnostic_plots(custchurn,i)
    

In [ ]:
# 3-3: Plot Categorical variables excluding 'int' and 'float' object datatype, and 'customerID'


cat_columns=custchurn.select_dtypes(exclude=['int64','float']).columns
cat_columns=cat_columns.drop(['customerID'])

for i in cat_columns[:-1]:
    sns.catplot(x=i,y="tenure",hue="Churn",data=custchurn,kind="box",aspect=1);
    
    

### Step 4: Data preprocessing

In [8]:
#4-1: Replace target variable 'Churn' from 'Yes=1, 'No'=0, and drop 'customerID'

custchurn['Churn'].replace(to_replace='Yes', value=1, inplace=True)
custchurn['Churn'].replace(to_replace='No',  value=0, inplace=True)
custchurn=custchurn.drop(['customerID'], axis=1)


In [9]:
#4-2: Change 16 categorical to dummy variables

custchurn_dummy = pd.get_dummies(custchurn)
print(f"Dataset Total records, Total variables: {custchurn_dummy.shape}")
#custchurn_dummy.head()

Dataset Total records, Total variables: (7043, 47)


In [10]:
#4-3: Normalise 3 numerical variables
#from sklearn.preprocessing import StandardScaler

num_cols = ["tenure", 'MonthlyCharges', 'TotalCharges']

scaler= StandardScaler()
custchurn_dummy[num_cols] = scaler.fit_transform(custchurn_dummy[num_cols])
custchurn_dummy.head(5)

,tenure,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,-1.277445,-1.160323,-0.992611,0,1,0,1,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0
1,0.066327,-0.259629,-0.172165,0,0,1,1,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1
2,-1.236724,-0.362660,-0.958066,1,0,1,1,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1
3,0.514251,-0.746535,-0.193672,0,0,1,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0
4,-1.236724,0.197365,-0.938874,1,1,0,1,0,1,0,1,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0


In [ ]:
# 4-4: Plot Correlation of 'Churn' with other variables

corr = custchurn_dummy.corr()

plt.figure(figsize=(15,8))
corr['Churn'].sort_values(ascending = False).plot(kind='bar')


In [ ]:
# 4-5: Plot Heatmap of Correlation of 'Churn' with other variables

mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
# Heatmap
plt.figure(figsize=(15, 10))
sns.heatmap(corr,
            vmax=.5,
            mask=mask,
            #annot=True, fmt='.2f',
            linewidths=.2, cmap="YlGnBu")


In [11]:
#4-6: Split to test and train dataset 70:30 ratio, check ratio of 0 and 1 in target variable

x=custchurn_dummy.drop(['Churn'], axis=1)
y=custchurn_dummy.Churn
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=0)

print("x_train, x_test:", x_train.shape, x_test.shape)
print("y_train, y_test:", y_train.shape, y_test.shape)
print(y_train.value_counts(), '\n', y_test.value_counts())


x_train, x_test: (4930, 46) (2113, 46)
y_train, y_test: (4930,) (2113,)
0    3614
1    1316
Name: Churn, dtype: int64 
 0    1560
1     553
Name: Churn, dtype: int64


In [ ]:
#4-7: Use SMOTE oversampling for unbalance training dataset

smt = SMOTE()
x1_train, y1_train = smt.fit_sample(x_train, y_train)
y1_train = y1_train.astype(int)

print("x1_train", x1_train.shape)
print("y1_train", y1_train.shape)
print(y1_train.value_counts())


### Step 4: Model Training and Validation 

In [15]:
# 4-1: The shallow learning models to be used
def get_models():
    models = list()
    models.append(GradientBoostingClassifier())
    models.append(DecisionTreeClassifier())
    models.append(RandomForestClassifier())
    models.append(LogisticRegression(max_iter=1000))
    models.append(GaussianNB())
    models.append(ExtraTreesClassifier())
    models.append(BaggingClassifier())
    models.append(AdaBoostClassifier())
    models.append(KNeighborsClassifier())
    models.append(SVC(kernel = "linear"))
    models.append(SVC(kernel = "rbf"))
    models.append(SVC(kernel = "poly"))
    models.append(SVC(kernel = "sigmoid"))
    return models


### New : k-fold cross validation

In [16]:
#4-2a: Training with k-fold cross validation of train dataset, and test dataset

# define test conditions
cv = KFold(n_splits=10, shuffle=True, random_state=1)

# evaluate model with a given test condition
def evaluate_model(cv, model):
    start1=time.time()
    scores_a = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
    scores_r = cross_val_score(model, x, y, scoring='recall', cv=cv, n_jobs=-1)
    val_time=time.time()-start1
    scores_a1 = float("{0:.3f}".format(np.mean(scores_a)))
    scores_r1 = float("{0:.3f}".format(np.mean(scores_r)))
    val_time1 = float("{0:.3f}".format(val_time))
    #print(model,":  " ,"Accuracy =", float("{0:.3f}".format(np.mean(scores_a))), "Recall =", float("{0:.3f}".format(np.mean(scores_r))), "Validation Time=", float("{0:.3f}".format(val_time)))
    print("Validating ", model)
    return model, scores_a1, scores_r1, val_time1 
                      
# get the list of models to consider
models = get_models()

# collect results
cv_results = list()
results = pd.DataFrame()

# evaluate each model
for model in models:
    cv_mean = evaluate_model(cv, model)
    cv_results.append(cv_mean)

# print and save results as .csv
results = pd.DataFrame (cv_results,columns=['Model','val_acc','val_recall','val_time'])
print(results)
results.to_csv(r"C:\\Users\\wongannnee\\Google Drive\\results1_kfold.csv", index = False)


Validating  GradientBoostingClassifier()
Validating  DecisionTreeClassifier()
Validating  RandomForestClassifier()
Validating  LogisticRegression(max_iter=1000)
Validating  GaussianNB()
Validating  ExtraTreesClassifier()
Validating  BaggingClassifier()
Validating  AdaBoostClassifier()
Validating  KNeighborsClassifier()
Validating  SVC(kernel='linear')
Validating  SVC()
Validating  SVC(kernel='poly')
Validating  SVC(kernel='sigmoid')
                                Model  val_acc  val_recall  val_time
0        GradientBoostingClassifier()    0.801       0.521    34.029
1            DecisionTreeClassifier()    0.722       0.499     0.840
2            RandomForestClassifier()    0.790       0.488    10.360
3   LogisticRegression(max_iter=1000)    0.805       0.553     1.823
4                        GaussianNB()    0.696       0.843     0.348
5              ExtraTreesClassifier()    0.770       0.472    13.589
6                 BaggingClassifier()    0.776       0.458     4.137
7          

### New: Parameter Tuning for Logistics Regression

In [13]:
# 4-2b: Logistic Regression Parameter Tuning
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
penalty = ['l2','none']
c_values = [100, 10, 1.0, 0.1, 0.01]
class_weight = ['balanced','']

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values,class_weight=class_weight)
cv = KFold(n_splits=10, shuffle=True, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(x, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    

C:\Users\wongannnee\AppData\Local\Continuum\anaconda3\envs\scraper\lib\site-packages\sklearn\linear_model\_logistic.py:1321: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


Best: 0.806329 using {'C': 100, 'class_weight': '', 'penalty': 'none', 'solver': 'newton-cg'}
0.745275 (0.013111) with: {'C': 100, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'newton-cg'}
0.745275 (0.013111) with: {'C': 100, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'lbfgs'}
0.745275 (0.013111) with: {'C': 100, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}
0.745275 (0.013111) with: {'C': 100, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'sag'}
0.745275 (0.013249) with: {'C': 100, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'saga'}
0.745133 (0.012986) with: {'C': 100, 'class_weight': 'balanced', 'penalty': 'none', 'solver': 'newton-cg'}
0.745133 (0.012986) with: {'C': 100, 'class_weight': 'balanced', 'penalty': 'none', 'solver': 'lbfgs'}
0.000000 (0.000000) with: {'C': 100, 'class_weight': 'balanced', 'penalty': 'none', 'solver': 'liblinear'}
0.745133 (0.012986) with: {'C': 100, 'class_weight': 'balanced', 'penalty': 'no